In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
import glob
import yaml
from scipy.stats import spearmanr
from sklearn.metrics import auc
from tqdm import tqdm
import plotnine as p9
import pandas as pd
import numpy as np
import glob
import yaml
import shutil
from plotnine_prism import *
import sys
sys.path.append('../')
from src.utils import load_data
from src.utils import bootstrapping
from src.utils import compute_pearson_top_n
from src.utils import compute_area_under_pearson_top_n
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from scipy.stats import bootstrap
import numpy as np

In [ ]:
def normalize(x_list):
    x_scaled = scale(x_list)
    return x_scaled


def bootstrapping(x_list, n_resamples=1000):
    res = bootstrap((x_list,), np.median, n_resamples=n_resamples)
    standard_error = res.standard_error
    median = np.median(res.bootstrap_distribution)
    return [median, standard_error]

def transform_num(x):
    value = x.model.split("_")[-1]
    
    hparam = x.hparam
    
    if value.isnumeric():
        value = int(value)
    elif "." in value:
        value = float(value)
    else:
        pass
    
    return value
out_folder = "out_ablation"

In [ ]:
with open("../config.yaml", "r") as stream:
    DATASET_INFO = yaml.safe_load(stream)
DATASET_INFO

In [ ]:
with open(f"../10x_TuPro/config_dataset_a.yaml", "r") as stream:
        config_dataset = yaml.safe_load(stream)

models = config_dataset["MODEL"]
models

In [ ]:
datasets = ["USZ", "COAD", "10x_TuPro", "SCCRCC"] #  "10x_TuPro",  "SCCRCC"
datasets

In [ ]:
all_scores = []
for dataset in tqdm(datasets):
    
    genes = pd.read_csv(f"../{dataset}/{out_folder}/info_highly_variable_genes.csv")
    selected_genes_bool = genes.isPredicted.values
    genes_predict = genes[selected_genes_bool]
    scores = []
    for model in models:
    
        top_model = pd.read_csv(f"../{dataset}/{out_folder}/evaluation/{model}/top_model_per_test_sample.csv")
        model_dict = top_model[["test_sample", "model"]].set_index("test_sample").to_dict()['model']
    
        
        for test in model_dict.keys():
        
            file = f"../{dataset}/{out_folder}/evaluation/{test}/gene_scores/test/{model}/{model_dict[test]}.csv"
            score = pd.read_csv(file)
            score = score.set_index("index")
            score.columns = ["spearman"]
            score["test_sample"] = test
            score["model"] = model
            scores.append(score)
    
    
    scores = pd.concat(scores)
    scores["gene"] = scores.index
    scores = scores.reset_index(drop=True)
    scores = compute_pearson_top_n(scores, "model", genes_predict)
    scores["dataset"] = dataset
    all_scores.append(scores)

all_scores = pd.concat(all_scores)
all_scores = all_scores.groupby(["gene", "model", "top_n", "dataset"]).spearman.agg("mean").reset_index()
all_scores

In [ ]:
all_scores["hparam"] = all_scores.model.apply(lambda x: "\n".join(x.split("_")[1:-1]) if len(x.split("_")) > 1 else x)
all_scores["hparam_values"] = all_scores.model.apply(lambda x: int(x.split("_")[-1]) if x.split("_")[-1].isnumeric() or x.split("_")[-1] == "-1" else "full" if len(x.split("_")) == 1 else x.split("_")[-1])
tab = all_scores.copy()
tab

In [ ]:
auc_scores = []
for dataset in tab.dataset.unique():
    t = tab.query(f'dataset == "{dataset}"')
    res = compute_area_under_pearson_top_n(t, "model", "spearman")
    res['dataset'] = dataset
    auc_scores.append(res)
tab = pd.concat(auc_scores)
tab

In [ ]:
from scipy.stats import gmean
tab["hparam"] = tab.model.apply(lambda x: " ".join(x.split("_")[1:-1]) if len(x.split("_")) > 1 else x)
tab["hparam_values"] = tab.apply(lambda x: transform_num(x), axis=1)
#tab["auc_mean"] = tab.groupby(["hparam", "dataset"]).auc_mean.transform(lambda x: normalize(x))
#tab["auc_std"] = tab.groupby(["hparam", "dataset"]).auc_std.transform(lambda x: x - gmean(x))
tab.head()

In [ ]:
tab.hparam_values.unique()

In [ ]:
tab.hparam_values.unique()

In [ ]:
hparam_of_int = {'context': "Context", 
                 'oversampling': "Oversampling", 
                 'genenorm': "Normalization",
                 'image': 'Image model',
                 'loss': 'Loss'
                 }
values_of_int = {"off": "off", "on": "on",
                 "no": "no", 'maxabs': 'maxabs', 'robust': 'robust', 'standard': 'standard',
                 "default": "default", "aestetik": "aestetik",
                 "spot": "spot", "spotsubspotneighbors1": "+spatial\n1-hop neighb", "spotsubspotneighbors3": "+spatial\n3-hop neighb", "spotsubspotneighbors5": "+spatial\n5-hop neighb",
                 "inception": "inception", "resnet50": "resnet", "phikon": "phikon", "phikonv2": "phikon_v2", "uni": "uni", 'hoptimus0': 'hoptimus0',
                 'mse': 'mse', 'pearson': 'pearson'
                 }

In [ ]:
scace_facet = [max(0.4, dict(tab[["hparam", "hparam_values"]].drop_duplicates().hparam.value_counts() / 8)[a]) for a in hparam_of_int if a in tab.hparam.unique()]
scace_facet

In [ ]:
plot_data = tab.copy()
plot_data = plot_data.query("hparam in @hparam_of_int.keys()")
plot_data.hparam = plot_data.hparam.apply(lambda x: hparam_of_int[x])
plot_data.hparam_values = plot_data.hparam_values.apply(lambda x: values_of_int[x] if x in values_of_int else x)
order = values_of_int.values()
position_dodge_width = 0.6
plot_data["Dataset"] = plot_data.dataset.apply(lambda x: DATASET_INFO["DATASET_NAME"][x])
#plot_data.Dataset = pd.Categorical(plot_data.Dataset, DATASET_INFO["DATASET_NAME"].values())
plot_data.hparam = plot_data.hparam.astype(str)
plot_data.hparam = pd.Categorical(plot_data.hparam.values, hparam_of_int.values()).astype(str)
plot_data.hparam_values = pd.Categorical(plot_data.hparam_values.astype(str), order)

#plot_data["group"] = plot_data.top_n.astype(str) + plot_data.Dataset

p = (p9.ggplot(plot_data, p9.aes("hparam_values", "auc_mean")) 
 + p9.geom_point(p9.aes(group="Dataset"), 
                 size=1, 
                 position=p9.position_dodge(width=position_dodge_width))
 #+ p9.geom_line(p9.aes(color="Dataset", group="Dataset"), 
 #               linetype="dashed", 
 #               position=p9.position_dodge(width=position_dodge_width))
     #+ p9.geom_boxplot(p9.aes(color="Dataset"))
 + p9.theme_bw()
# + p9.facet_grid("~hparam", scales="free", space={"x": scace_facet, "y":[1]})
 + scale_color_prism(palette = "colors")
 + p9.ylab("AU Gene pearson correlation")
 + p9.xlab("Hyperparameter value")
 + p9.facet_wrap("~Dataset+hparam", nrow=len(datasets), scales='free',  drop=True)
 + p9.geom_errorbar(p9.aes(x="hparam_values", ymin="auc_mean-auc_std",ymax="auc_mean+auc_std"), 
                    width=0.1, alpha=1, size=1,
                     position=p9.position_dodge(width=position_dodge_width)
                   )
 + p9.theme(panel_spacing_y=-0.06, 
            panel_spacing_x=0, 
            figure_size=(32, 20), 
            axis_text_x = p9.element_text(angle = 90),
            text=p9.element_text(size=20),
            strip_text=p9.element_text(size=20),
            legend_title=p9.element_text(size=20),
            legend_text=p9.element_text(size=20))
)
p.save("figures/FigureS1-ablation.png", dpi=300,limitsize=False)
p

In [ ]:
1